In [1]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

In [11]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import *

In [2]:
from django.http import JsonResponse, HttpResponse, HttpResponseBadRequest
from myapi import serializers as myserializer
from django.db.models import Q          # untuk filter multi parameter

In [3]:
from myapi.models import *
from myapi.modulku import *
from myapi.serializers import *
from myapi.forms import *

In [4]:
from django.core.exceptions import ObjectDoesNotExist, ValidationError, FieldDoesNotExist

<center><h2 style="color: #0000FF; font-size: 40px">Halaman Simulasi</h2></center>

In [6]:
mapel = Mapel.objects.all()

In [9]:
for value in mapel:
    print(value.id)

1


In [10]:
arsip = Arsip.objects.all()

In [13]:
mapel_satu = Mapel.objects.get(pk=1)

In [15]:
mapel_detail = mapel_satu.arsip_set.all()

In [18]:
for value in mapel_detail:
    print(value.ke)

AttributeError: 'Arsip' object has no attribute 'myfile'

In [12]:
for value in arsip:
    print(value.mapel)

113


<center><h2 style="color: #FF0000; font-size: 40px">==== end =======</h2></center>

<center><h2 style="color: #0000FF; font-size: 40px">Halaman Data</h2></center>

In [24]:
# page = request.GET.get('page', 1)
# alamat = request.build_absolute_uri()

page = 3
alamat = 'mantap'

In [25]:
anggota = Anggota.objects.all()
anggota_serializer = AnggotaSerializer(anggota, many=True)

# paginasinya
anggota_paginasi = paginasinya(anggota_serializer.data, page, 10, alamat=alamat)

# return JsonResponse(anggota_paginasi, safe=False)

# 2 code dibawah ini jagnan dicopy di views.py
res = JsonResponse(anggota_paginasi, safe=False)
res.content

b'{"count": 1, "current": 3, "previous_page": null, "next_page": false, "previous": "0", "next": false, "total_data": 3, "data": [{"id": 2, "nama": "Ayu", "alamat": "Dongkala", "status": "Anggota"}, {"id": 3, "nama": "Mantap", "alamat": "PASARWAJO", "status": "ANGGOTA"}, {"id": 9, "nama": "Di buat di postman", "alamat": "Saragi", "status": "anggota"}], "alamat": "mantap"}'

<center><h2 style="color: #FF0000; font-size: 40px">end Halaman Data</h2></center>

<center><h2 style="color: #0000FF; font-size: 40px">halaman detail</h2></center>

In [13]:
# detail
id =2

In [15]:
anggotaDetail = Anggota.objects.get(pk=id)
anggota_serializer = AnggotaSerializer(anggotaDetail)

try:
    profile_anggota = anggotaDetail.profile
    foto_anggota = anggotaDetail.foto_set.all()   
    profile_serializer = FotoSerializer(profile_anggota)
    
    context = {
        "anggota": anggota_serializer.data,
        "profile": profile_serializer.data
    }
except ObjectDoesNotExist:
    profile_anggota = {
        "keterangan" : "",
        "selfi" : "",
        "ktp" : "",
        "diri" : "",
        "anggota" : None,
        "id": None,
    }
    
    context = {
        "anggota": anggota_serializer.data,
        "profile": profile_anggota
    }
    
# return JsonResponse(context)      # tanpa paginas
    
# 2 code di bawah ini dicomment ketika dipaste di views.py
res = JsonResponse(context)
res.content

b'{"anggota": {"id": 2, "nama": "Ayu", "alamat": "Dongkala", "status": "Anggota"}, "profile": {"id": 17, "keterangan": "ini keterangan yang di rubah di postman id 2", "selfi": null, "ktp": null, "diri": null, "anggota": 2}}'

<center><h2 style="color: #FF0000; font-size: 40px">end halaman detail</h2></center>

<center><h2 style="font-size: 40px; color: #0000FF">Post Data</h2></center>

<h2>======================= simpan data baru ==================================</h2>

In [41]:
# if method.request == 'POST':
data_post = {
    "nama": "Kurnia",
    "alamat": "KOMBELI",
    "status": "ANGGOTA"
}

In [42]:
anggota_form = AnggotaForm(data_post or None)
anggota_form.is_valid()

True

In [43]:
# simapan data
anggota_form.save()

<Anggota: Kurnia>

In [30]:
# ini untuk mengecek error
# anggota_form.errors

<h2>======================= update data yang ada ==============================</h2>

In [57]:
data_profile = {
    "keterangan": "ini adalah id 3",
    "selfi": None,
    "diri": None,
    "ktp": None,
    "anggota": 3 
}

In [46]:
id = 5

In [48]:
profile_anggota = Profile.objects.get(pk=id)
form_update_profile = ProfileForm(data_profile or None, instance=profile_anggota)

In [49]:
# untuk mengecek apakah sesuai atau tidak
form_update_profile.is_valid()

True

In [28]:
# untuk mengecek ada error atau tidak
# form_update_profile.errors

In [50]:
form_update_profile.save()

<Profile: Profile object (5)>

In [13]:
'''=============================== modifikasi sebelum simpan ======================================================'''
modifikasi = form_update_profile.save(commit=False)
modifikasi.anggota_id = 3
modifikasi.keterangan = "oke dirubah lah sudah berhasil dan bagus'"
# modifikasi.anggota =3
modifikasi.ktp = None
'''=============================================================================================================='''

'=============================================================================================================='

In [14]:
form_update_profile.save()

<Profile: Profile object (2)>

In [22]:
data_profile = {
    "keterangan": "ini adalah id dirubajfdfjdsfjsdfjsdfjsdfj3",
    "selfi": None,
    "diri": None,
    "ktp": None,
    "anggota": 3
}

'''================================= update data relasi =================================================================='''
id = 3
d_anggota = Anggota.objects.get(pk=id)
d_profile = False
try:
    d_profile = Profile.objects.get(pk=d_anggota.profile.id)
    proses_simpan = ProfileForm(data_profile, instance=d_profile)
    if proses_simpan.is_valid():
        proses_simpan.save(commit=False)
        proses_simpan.anggota = id
        proses_simpan.save()
        print('save')
except Profile.DoesNotExist:
    proses_simpan = ProfileForm(data_profile)
    if proses_simpan.is_valid():
        proses_simpan.save(commit=False)
        proses_simpan.anggota = id
        proses_simpan.save()
        print('buat baru')
'''====================================================================================================================='''

save


'====================================================================================================================='

<h2>======================= delete data yang ada ==============================</h2>

In [44]:
data_anggota = Anggota.objects.all()
data_anggota.values()

<QuerySet [{'id': 1, 'nama': 'Bambang', 'alamat': 'Jakarta', 'status': 'Anggota'}, {'id': 2, 'nama': 'Ayu', 'alamat': 'Dongkala', 'status': 'Anggota'}, {'id': 3, 'nama': 'Mantap', 'alamat': 'PASARWAJO', 'status': 'ANGGOTA'}, {'id': 4, 'nama': 'Kumbang', 'alamat': 'Takimpo', 'status': 'Anggota'}, {'id': 5, 'nama': 'nama Jupyter', 'alamat': 'KOMBELI', 'status': 'ANGGOTA'}, {'id': 7, 'nama': 'Kurnia', 'alamat': 'KOMBELI', 'status': 'ANGGOTA'}]>

In [45]:
profile_anggota.values()

<QuerySet [{'id': 1, 'keterangan': 'update keterangan yang id satu dan mantap jiwa', 'selfi': '', 'ktp': '', 'diri': '', 'anggota_id': 2}, {'id': 2, 'keterangan': "oke dirubah lah sudah berhasil dan bagus'", 'selfi': '', 'ktp': '', 'diri': '', 'anggota_id': 3}, {'id': 3, 'keterangan': 'telah di ubah', 'selfi': '', 'ktp': '', 'diri': '', 'anggota_id': 1}, {'id': 5, 'keterangan': None, 'selfi': '', 'ktp': '', 'diri': '', 'anggota_id': 7}]>

In [35]:
# inputan
pk = 6

In [37]:
data_anggota_hapus = Anggota.objects.get(id=pk)
data_anggota_hapus

<Anggota: Kurnia>

In [38]:
# preses menghapus
data_anggota_hapus.delete()

(2, {'myapi.Profile': 1, 'myapi.Anggota': 1})

In [ ]:
''' 
pengkondisian 

kondisi = True
if kondisi:
    return JsonResponse({"pesan": "sukses hapus"}, status=200)

return JsonResponse({"pesan": "gagal hapus"}, status=400)
'''



<h2 style="color: #0000ff;">delete multiple data selected</h2>

In [113]:
# delete multiple
'''

if request.method == 'POST':
    anggota_id = request.POST.getlist('id[]')
    for value in anggota_id:
        anggota = Anggota.objects.get(id=value)
        anggota.delete()
    
    return redirect('')   # diarahkan kembali ke halaman data tampil anggota
'''

"\n\nif request.method == 'POST':\n    anggota_id = request.POST.getlist('id[]')\n    for value in data_id:\n        anggota = Anggota.objects.get(id=value)\n        anggota.delete()\n    \n    return redirect('')   # diarahkan kembali ke halaman data tampil anggota\n"

<h2>=========================== Filter Data ==================================</h2>

In [93]:
katakunci = "kombb"

In [94]:
filter_data_anggota = Anggota.objects.filter(Q(nama__contains=katakunci) | Q(alamat__contains=katakunci))

In [95]:
hasilcari_serializer = AnggotaSerializer(filter_data_anggota, many=True)

# paginasinya
'''
page = request.GET.get('page', 1)
alamat = request.build_absolute_uri()
'''

page = 3
alamat = 'mantap'

anggota_paginasi = paginasinya(hasilcari_serializer.data, page, 10, alamat=alamat)

# return JsonResponse(anggota_paginasi, safe=False)

# 2 code dibawah ini jagnan dicopy di views.py
hasilcari = JsonResponse(anggota_paginasi, safe=False)
hasilcari.content

b'{"count": 1, "current": 3, "previous_page": null, "next_page": false, "previous": "0", "next": false, "total_data": 0, "data": [], "alamat": "mantap"}'

<h2 style="color: #ff0000"><center>End Post</center></h2>

<h2 style="color: #0000ff"><center>queryset (request) post atau get </center></h2>

In [1]:
from django.http import QueryDict

In [ ]:
<QueryDict: {'id[0]': ['21'], 'id[1]': ['18'], 'id[2]': ['19'], 'id[3]': ['16']}>

In [6]:
def query(x):
    query_dictionary = QueryDict('', mutable=True)
    query_dictionary.update(x)
    return query_dictionary

In [81]:
dataku6 = {
    id: ["4","7","10"]
}

In [21]:
dataku = {
    "id[0]": 5,
    "id[1]": 2, 
    "id[2]": 31,
}

In [58]:
dataku3 = {
    "id[]": 3,
    "id[]": 1,
    "id[]": 29
}

In [69]:
dataku4 = {
    "id": 44,
    "id": 10,
    "id": 50
}

In [74]:
dataku5 = {
    "id": [40, 17, 19]
}

In [82]:
idku6 = query(dataku6)
idku6

<QueryDict: {<built-in function id>: [['4', '7', '10']]}>

In [75]:
idku5 = query(dataku5)
idku5

<QueryDict: {'id': [[40, 17, 19]]}>

In [85]:
idku6.getlist('id[]')

[]

In [80]:
idku5.getlist('id')[0]

[40, 17, 19]

In [70]:
idku4 = query(dataku4)

In [73]:
idku4.getlist('id[]')

[]

In [60]:
idku3 = query(dataku3)

In [68]:
idku3

<QueryDict: {'id[]': [29]}>

In [22]:
idku = query(dataku)
idku

<QueryDict: {'id[0]': [5], 'id[1]': [2], 'id[2]': [31]}>

In [57]:
idku['id[1]']

2

In [50]:
idku2 = query(dataku2)
idku2

<QueryDict: {<built-in function id>: [[4, 7, 10]]}>

In [34]:
idku.getlist('id')

[]

In [52]:
idku2.getlist('id[dd]')

[]

In [42]:
paramatersnya = idku.getlist('id[]')

In [46]:
paramatersnya

[]

In [47]:
ab = [1,2,3,5]

In [48]:
ab

[1, 2, 3, 5]

In [33]:
for v in idku.getlist('id[]'):
    print('saya')
    print(v)

In [2]:
ordinary_dict = {
    "a": "ini adalah a",
    "b": "ini adalah nilai b",
    "id": ['[2,4]'],
    "pk": [2,4]
}

In [3]:
query_dict = QueryDict('', mutable=True)

In [4]:
query_dict.update(ordinary_dict)

In [5]:
query_dict['id']

['[2,4]']

In [163]:
query_dict.getlist('get')

[]

In [10]:
data_anggota = Anggota.objects.get(id=3)
data_anggota.profile

<Profile: Profile object (19)>

In [106]:
idku.dict()

{'id[0]': 5, 'id[1]': 2, 'id[2]': 31}

In [110]:
idku.getlist('id[]')

[]

In [114]:
idku

<QueryDict: {'id[0]': [5], 'id[1]': [2], 'id[2]': [31]}>

In [113]:
for index, item in enumerate(idku):
    print(index, item)

0 id[0]
1 id[1]
2 id[2]


In [121]:
for index, item in idku.getlist('id[]'):
    print(index, item)

In [138]:
for row in idku.getlist('id[]'):
    print(row)

In [151]:
for key, value in idku.items():
    print(value)

5
2
31


In [163]:
datasaya = {
    "id": 1,
    "id": 4,
    "id": 5
}

In [164]:
d = query(datasaya)

In [167]:
d.getlist()

<QueryDict: {'id': [5]}>

In [161]:
d

<QueryDict: {'id': [[1, 10, 9]]}>

In [152]:
idku

<QueryDict: {'id[0]': [5], 'id[1]': [2], 'id[2]': [31]}>

In [ ]:
for key, value idku.g

In [127]:
idku.get('id[2]')

31

In [135]:
idku.getlist('id[2]')

[31]

In [143]:
for k, v in idku.items():
    print(v)

5
2
31


In [2]:
import requests

In [3]:
alamatnya = "https://www.google.com?nama=3"

In [5]:
requests(alamatnya)

TypeError: 'module' object is not callable

In [8]:
requests.utils.quote('koe dan kamu')

'koe%20dan%20kamu'

In [13]:
postman = requests.get('http://127.0.0.1:8000/myapi/')

In [16]:
postman_create = requests.post('http://127.0.0.1:8000/myapi/create_data')

In [17]:
postman_create.content

b'{"count": 1, "current": 1, "previous_page": null, "next_page": false, "previous": "0", "next": false, "total_data": 6, "data": [{"id": 9, "nama": "Di buat di postman", "alamat": "Saragi", "status": "anggota"}, {"id": 11, "nama": "nama dua", "alamat": "alamat dua", "status": "anggota"}, {"id": 12, "nama": "nama tiga", "alamat": "alamat tiga", "status": "anggota"}, {"id": 15, "nama": "nama enam", "alamat": "alamat  enam", "status": "anggota"}, {"id": 16, "nama": "nama tujuh", "alamat": "alamat tujuh", "status": "naggota"}, {"id": 17, "nama": "nama delapan", "alamat": "alamat delapan", "status": "anggota"}], "safe": false, "alamat": "http://127.0.0.1:8000/myapi/"}'

In [14]:
postman.headers

{'Date': 'Fri, 03 Sep 2021 06:16:38 GMT', 'Server': 'WSGIServer/0.2 CPython/3.8.3', 'Content-Type': 'application/json', 'X-Frame-Options': 'DENY', 'Content-Length': '669', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff', 'Referrer-Policy': 'same-origin'}

In [15]:
postman.content

b'{"count": 1, "current": 1, "previous_page": null, "next_page": false, "previous": "0", "next": false, "total_data": 6, "data": [{"id": 9, "nama": "Di buat di postman", "alamat": "Saragi", "status": "anggota"}, {"id": 11, "nama": "nama dua", "alamat": "alamat dua", "status": "anggota"}, {"id": 12, "nama": "nama tiga", "alamat": "alamat tiga", "status": "anggota"}, {"id": 15, "nama": "nama enam", "alamat": "alamat  enam", "status": "anggota"}, {"id": 16, "nama": "nama tujuh", "alamat": "alamat tujuh", "status": "naggota"}, {"id": 17, "nama": "nama delapan", "alamat": "alamat delapan", "status": "anggota"}], "safe": false, "alamat": "http://127.0.0.1:8000/myapi/"}'

In [1]:
import requests

In [8]:
alamatnya = 'http://127.0.0.1:8000/myapi/coba?id=1&id=2&id=5&nama=mantap&nama=ladua&nama=latiga'

In [9]:
alamat_coba = requests.get(alamatnya)

In [6]:
alamat_coba.content

b'pesan'

<h2>authenticated</h2>

In [1]:
 from django.contrib.auth import models

In [5]:
models.User.is_authenticated

In [14]:
semuadata = models.User.objects.all()

In [16]:
for value in semuadata:
    print(value.id)

6
7
8
9
10
11
12
13


In [17]:
lasepuluh = models.User.objects.get(pk=6)

In [31]:
lasepuluh.profil

4

In [24]:
if requ.is_authenticated:
	akun = request.user.

False

In [20]:
Profile.objects.filter(anggota_id=None).delete()

(3, {'myapi.Profile': 3})

In [32]:
from myapi.forms import *

In [57]:
inputan = {
    "user": 7,
    "nama_lengkap": "telah diubah",
    "phone": 234,
    "nim": 1324,
    "alamat": "alamat telah diubah",
    "email": "mantap@gmail.com"
}

In [58]:
profil_id = Profil.objects.get(pk=11)

In [63]:
myform = ProfilForm(inputan, instance=profil_id)

In [79]:
profil_id.nama_lengkap

'telah diubah'

In [73]:
myform.instance.alamat

'alamat telah diubah'

In [60]:
myform.is_valid()

False

In [61]:
myform.errors

{'user': ['Profil with this User already exists.']}

In [102]:
inputan_profil = {
    "user": "7",
    "nama_lengkap": "telah diubah",
    "phone": 234,
    "nim": 1324,
    "alamat": "alamat telah diubah",
    "email": "mantap@gmail.com"
}

In [83]:
mapel_inputan = {
    "kode_pelajaran": "212212",
    "nama_pelajaran": "Matematika Diskrit"
}

In [89]:
user_id = User.objects.get(pk=7)

In [92]:
user_id.profil.id

5

In [98]:
mapel_id = Mapel.objects.get(pk=5)
profil_id = Profil.objects.get(pk=5)

In [103]:
form_mapel = MapelForm(mapel_inputan, instance=mapel_id)
form_profil = MapelForm(inputan_profil, instance=profil_id)

In [86]:
form_mapel.is_valid()

True

In [107]:
form_profil.errors

{}

In [104]:
form_profil.is_valid()

True

In [105]:
form_profil.save()

<Profil: Profil object (5)>

In [1]:
from import_export import resources

In [3]:
class Myresource(resources.ModelResource):
    nama = resources.

In [5]:
Myresource

__main__.Myresource

In [106]:
profil_id.alamat

In [10]:
a = [1, 3, 6, 4, 6, 6]
b = [1, 3, 6, 4, 6, 6]

In [21]:
precision_score(a, b, average='weighted')

1.0

In [16]:
a = ["II", "III", "II", "II", "III", "II"]
b = ["II", "III", "II", "II", "III", "II"]

In [9]:
classification_report(a, b)

'              precision    recall  f1-score   support\n\n          II       1.00      1.00      1.00         4\n         III       1.00      1.00      1.00         2\n\n    accuracy                           1.00         6\n   macro avg       1.00      1.00      1.00         6\nweighted avg       1.00      1.00      1.00         6\n'